In [1]:
import re
import json
import gzip
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import torch
import tensorflow as tf

from transformers import T5ForConditionalGeneration, PreTrainedTokenizerFast

In [2]:
json_test_data_path = "melik_custom_noised_test_cases.tar.gz"
gzip_file = gzip.open(json_test_data_path)
byte_lines = [line for line in gzip_file]
json_lines = [json.loads(byte_line) for byte_line in byte_lines]
print('Number of Test Cases:', len(json_lines))

def pretty_print(ground_truth, model_output):
    """
    - For both Ground Truth and Model Output:
        - count the number of <extra_id_{idx}> tokens.
        - iterate them and get the start and end locations of each.
        - print between <extra_id_{idx}> and <extra_id_{idx+1}>
    """

    # Count the number of sentinel tokens
    num_sentinel_tokens = len(re.findall('<extra_id_', model_output))
    sentinel_token_locs = [] #((ground_truth_start, ground_truth_end), (model_output_start, model_output_end))
    # Extract sentinel token locations
    for idx in range(num_sentinel_tokens):
        sentinel_token = f'<extra_id_{idx}>'
        sentinel_token_len = len(sentinel_token)
    
        # Ground Truth
        start_loc_in_ground_truth = ground_truth.find(sentinel_token)
        end_loc_in_ground_truth = start_loc_in_ground_truth + sentinel_token_len
    
        # Model Output
        start_loc_in_model_output = model_output.find(sentinel_token)
        end_loc_in_model_output = start_loc_in_model_output + sentinel_token_len
    
        sentinel_token_locs.append(((start_loc_in_ground_truth, end_loc_in_ground_truth), (start_loc_in_model_output, end_loc_in_model_output)))

    # Print sentinel tokens 
    for idx in range(num_sentinel_tokens - 1):
        ground_truth_print = ground_truth[sentinel_token_locs[idx][0][1]:sentinel_token_locs[idx+1][0][0]]
        model_output_print = model_output[sentinel_token_locs[idx][1][1]:sentinel_token_locs[idx+1][1][0]]
        print(f'<extra_id_{idx}>', ground_truth_print, '|', model_output_print, f'<extra_id_{idx+1}>')

Number of Test Cases: 94


In [3]:
config_file = 'config.json'
#pytorch_total_params = sum(p.numel() for p in model.parameters())
#print(f'Number of Model Parameters: {pytorch_total_params:,}')
tokenizer = PreTrainedTokenizerFast.from_pretrained('VBARTTokenizer_T5_Sentinels')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

checkpoints = [file for file in os.listdir() if "converted_pt_model_" in file]
checkpoints = sorted(checkpoints, key = lambda x: int(x.split('_')[-1]))
print(checkpoints)
print('--------')

for json_line in json_lines[20:25]:
    model_input = json_line['inputs']
    model_input = model_input.replace(' <extra_id_', '<extra_id_') # Huggingface Tokenizer creates extra whitespace tokens. I prevent them here.
    ground_truth = json_line['targets']
    
    inputs_encoded = tokenizer.encode(model_input)
    model_inputs = tf.keras.utils.pad_sequences([inputs_encoded], maxlen = 512, padding = 'post', truncating = 'post')
    print('Model Input:\n', tokenizer.decode(model_inputs[0]))
    print('------------------------------------')
    model_inputs = torch.from_numpy(model_inputs).to(device)
    for checkpoint in checkpoints[-2:]:
        print('Checkpoint:', checkpoint)
        model = T5ForConditionalGeneration.from_pretrained(checkpoint)
        
        model_outputs = model.generate(inputs = model_inputs, max_length = 512, do_sample = False, num_beams = 1)
        model_output_as_list = model_outputs.numpy()[0].tolist()
        model_output = tokenizer.decode(model_output_as_list, skip_special_tokens = False)
        #print(model_output)
        pretty_print(ground_truth, model_output)
        print('=====================================')
    print('\n\n')

/home/meliksah.turker/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Device: cpu
['converted_pt_model_150000', 'converted_pt_model_250000', 'converted_pt_model_500000', 'converted_pt_model_1000000', 'converted_pt_model_1250000', 'converted_pt_model_1500000']
--------
Model Input:
 [NLU]nin dikkatli davranmadığını kanıtlamayan küçük fiziksel bir kötülüktür ve öyle kalır. Bu analiz daha başka bir sonuca vardır<extra_id_0> Sonra, insan elinden yediği bir darbeyi en büyük kötülük zanneden aynı şövalyenin,<extra_id_1> kat daha şiddetli bir darbeyi atından yediğini ve acıyla sendeleyip uzaklaşırken, bunun bir önemi olmadığını söylediğini görüyorum. O zaman, sorunun insan elinde<extra_id_2> , şövalyelerimizin dövüş sırasında bu eskrim<extra_id_3> ve kılıç darbelerini yediklerini ve bunların sözünü etmeye bile değmeyen, önemsiz olaylar<extra_id_4> üzerine,<extra_id_5> vurulan darbelerin bile, sopalarla vurulanlar<extra_id_6> kötü olmadığını gözlemliyorum, çünkü daha yakın zamanlarda<extra_id_7> askeri okul öğrencileri<extra_id_8> ikincilere maruz kalıyorlardı; 